<a href="https://colab.research.google.com/github/dmika1234/ml_uwr_22/blob/Project/Project/tfidvec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Models
from sklearn.linear_model import LogisticRegression

# Funs
# from funs import *

ModuleNotFoundError: No module named 'funs'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors


def join_fun(list, sep=' '):
    text = sep.join(list)
    return text


class DataPreprocessor:
    def __init__(self) -> None:
        self.tokenizer = RegexpTokenizer(r'\w+')
        nltk.download('stopwords') 
        nltk.download('wordnet')
        nltk.download('omw-1.4')
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
    def preprocess_data(self, text_data, column_names, vectorize_fun=list):
        text_data = text_data[column_names].copy()
        text_data.fillna('', inplace=True)
        
        for i in np.arange(text_data.shape[0]):
            for col in column_names:
                text_data.at[i,col] = self.tokenizer.tokenize(text_data.at[i,col].lower()) # removing punctuation
                filtered_sentence = []
                for w in text_data.at[i,col]:
                    if w not in self.stop_words: # removing words such as “the”, “a”, “an”, “in”
                        filtered_sentence.append(self.lemmatizer.lemmatize(w)) # grouping together the different inflected forms of a word 
                text_data.at[i,col] = vectorize_fun(filtered_sentence)

        return text_data


def get_train_test_indcs(X, y, test_size, random_state, stratify):
    X_train, X_test = train_test_split(X, y, test_size=test_size,
     random_state=random_state, stratify=stratify)[0:2]
    return X_train.index, X_test.index


def evaluate_performance(y_test, y_pred, prob=False, threshold=0.2, round=4, save_pred=False):
    if prob:
        y_pred = (y_pred >= threshold).astype('int64')
    results = {}
    if save_pred:
        results['y_pred'] = y_pred
    # Get the confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # Calculate the detection percentage
    results['detection_percentage'] = np.round(tp / (tp + fn), round)
    # Calculate precision
    results['precision'] = np.round(tp / (tp + fp), round)
    # Calculate accuracy
    results['accuracy'] = np.round((tp + tn) / (tp + tn + fp + fn), round)
    # Calculate F1-score
    results['f1_score'] = np.round(2 * (results['precision'] * results['detection_percentage']) /\
                          (results['precision'] + results['detection_percentage']), round)
    results['auc_roc'] = np.round(roc_auc_score(y_test, y_proba), round)

    return results

## Loading the data

In [2]:
# Locally
# data_url = 'https://github.com/dmika1234/ml_uwr_22/blob/Project/Project/data/fake_job_postings.csv'
data_path = 'data/fake_job_postings.csv'
raw_data = pd.read_csv(data_path)

# For colab
# data_url = '/content/fake_job_postings.csv'
# raw_data = pd.read_csv(data_url, error_bad_lines=False, engine="python")
#straszny problem miałem, żeby wczytać te dane tak ja ty to robiłeś. dziwne błędy mi wyskakiwały

## Preprocessing with nltk

In [2]:
text_colnames = ['company_profile', 'description', 'requirements', 'benefits']
DataPrep = DataPreprocessor()

# text_data_ls = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=list)
# text_data_np = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=np.array)
text_data_str = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=join_fun)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
working_df = raw_data.copy()

Spliting location into country, state, city

In [5]:
working_df[['country', 'state', 'city']] = working_df['location'].str.split(',', expand=True).iloc[:,0:3]

Splitting salary range into min, max salary

In [6]:
working_df[['salary_min', 'salary_max']] = working_df['salary_range'].str.split('-', expand=True)
working_df[['salary_min', 'salary_max']] = working_df[['salary_min', 'salary_max']].apply(pd.to_numeric, errors='coerce').fillna(0)

In [7]:
target_colname = 'fraudulent'
# Getting numerical colnames and deleting not useful
numerical_colnames = list(working_df.select_dtypes(include='int64').columns)
numerical_colnames = list(set(numerical_colnames) - set(['job_id', target_colname]))
numerical_colnames = numerical_colnames + ['salary_min', 'salary_max']
# Getting other text colnames and deleting not useful
other_text_colnames = list(set(working_df.select_dtypes(include='object').columns) - set(text_colnames))
other_text_colnames = list(set(other_text_colnames) - set(['location', 'salary_range']))
print(numerical_colnames)
print(text_colnames)
print(other_text_colnames)

['has_questions', 'telecommuting', 'has_company_logo', 'salary_min', 'salary_max']
['company_profile', 'description', 'requirements', 'benefits']
['industry', 'required_experience', 'title', 'state', 'employment_type', 'required_education', 'city', 'department', 'country', 'function']


Filling missing values

In [8]:
working_df[text_colnames + other_text_colnames] = working_df[text_colnames + other_text_colnames].fillna('')
working_df[numerical_colnames] = working_df[numerical_colnames].fillna(0)

In [9]:
working_df[other_text_colnames].apply(lambda x: np.unique(x).shape[0]).sort_values()

employment_type            6
required_experience        8
required_education        14
function                  38
country                   91
industry                 132
state                    326
department              1338
city                    2336
title                  11231
dtype: int64

We will only use those with not so much levels(<50 for start)

In [10]:
final_other_text_colnames = ['employment_type', 'required_experience', 'required_education', 'function']

In [11]:
X = pd.get_dummies(working_df[final_other_text_colnames], columns=final_other_text_colnames)
X[numerical_colnames] = working_df[numerical_colnames]
y = working_df[target_colname]

Splitting the data

In [13]:
train_indcs, test_indcs = get_train_test_indcs(raw_data, raw_data['fraudulent'],
 test_size=.1, random_state=42, stratify=raw_data['fraudulent'])

## Using TfidVectorizer to change text to numerical vectors

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

number_of_vars = 50
X_tfdif = pd.DataFrame()
for colname in text_colnames:
    vectorizer = TfidfVectorizer(max_features=number_of_vars, min_df=2).fit(text_data_str[colname][train_indcs])
    df_transformed = pd.DataFrame(vectorizer.transform(text_data_str[colname]).toarray(), 
    columns=['num_' + colname + '_' + str(nr) for nr in np.arange(number_of_vars)])
    print(f'{colname} data successfuly transformed!')
    X_tfdif = pd.concat((X_tfdif, df_transformed), axis=1)
X_tfdif.shape
X_tfdif_final = pd.concat((X, X_tfdif), axis=1)
X_tfdif_final.shape

company_profile data successfuly transformed!
description data successfuly transformed!
requirements data successfuly transformed!
benefits data successfuly transformed!


(17880, 271)

# Modeling

### Train-test split

In [16]:
X_tf_train, X_tf_test, y_train, y_test = X_tfdif_final.iloc[train_indcs], X_tfdif_final.iloc[test_indcs], y[train_indcs], y[test_indcs]

In [17]:
import httpimport

In [19]:
import httpimport
with httpimport.github_repo('dmika1234', 'ml_uwr_22', 
                            module='Project', branch='Project'):
     from funs import *

In [ ]:
evaluate_performance